In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import itertools
import numpy as np
import re
import pandas as pd
import math

In [2]:
def cleanAccent(s):
    s = s.replace("\u00e0", "a")
    s = s.replace("\u0103", "a")
    s = s.replace("\u00e1", "a")
    s = s.replace("\u00e2", "a")
    s = s.replace("\u0391", "a")
    s = s.replace("\u03b1", "a")
    
    s = s.replace("\u00c0", "A")
    s = s.replace("\u00c1", "A")
    s = s.replace("\u00c2", "A")
    s = s.replace("\u00c3", "A")
    s = s.replace("\u00c4", "A")
    s = s.replace("\u00c5", "A")
    s = s.replace("\u0102", "A")
    
    
    s = s.replace("\u00e9", "e")
    s = s.replace("\u00e8", "e")
    
    s = s.replace("\u00C8", "E")
    s = s.replace("\u00C9", "E")
    
    s = s.replace("\u00ed", "i")
    s = s.replace("\u00ec", "i")
    s = s.replace("\u00ee", "i")
    
    s = s.replace("\u00CC", "I")
    s = s.replace("\u00CD", "I")
    s = s.replace("\u00CE", "I")
    s = s.replace("\u013a", "I")
    
    s = s.replace("\u00f3", "o")
    s = s.replace("\u00f6", "o")
    s = s.replace("\u0151", "o")
    s = s.replace("\u039f", "o")
    s = s.replace("\u0151", "o")
    
    s = s.replace("\u00d2", "O")
    s = s.replace("\u00d3", "O")
    s = s.replace("\u00d6", "O")
    s = s.replace("\u00d4", "O")
    s = s.replace("\u0150", "O")
    
    s = s.replace("\u039b", "l")
    
    s = s.replace("\u00fa", "u")
    s = s.replace("\u0171", "u")
    s = s.replace("\u00fc", "u")
    s = s.replace("\u016b", "u")
    
    s = s.replace("\u00d9", "U")
    s = s.replace("\u00da", "U")
    s = s.replace("\u00db", "U")
    s = s.replace("\u00dc", "U") 
    s = s.replace("\u0170", "U")
    
    s = s.replace("\u015f", "s")
    s = s.replace("\u0219", "s")

    s = s.replace("\u015e", "S")
    s = s.replace("\u0218", "S")
    
    s = s.replace("\u021a", "t")
    s = s.replace("\u0163", "t")
    
    s = s.replace("\u021b", "T")
    s = s.replace("\u0162", "T")
    
    s = s.replace("\u0e4f", "")
    s = s.replace("\u032f", "")
    s = s.replace("\u0361", "")
    
    s = s.encode('ascii', 'ignore').decode('utf8')
    
    return s

In [18]:
url = "http://nepszamlalas.adatbank.ro/"
resp = requests.get(url)
html = resp.content
soup = BeautifulSoup(html)

In [19]:
#a legördülő listából a megyék mellé rendeljük a kódokat (csak az első 16 kell)
# az option jeloli a legordulo listaban megjeleno megyeket es megkapjuk a hozza tartozo kodot is 
option_tags = soup.find_all("option")

county_index = {}
for option in option_tags:
    if(int(option['value']) <17 and int(option['value']) > 0): 
        county_index[int(option['value'])] = option.text

county_index

{11: 'Alba (Fehér)',
 8: 'Arad',
 4: 'Bihor (Bihar)',
 12: 'Bistriţa-Năsăud (Beszterce-Naszód)',
 14: 'Braşov (Brassó)',
 9: 'Caraş-Severin (Krassó-Szörény)',
 13: 'Cluj (Kolozs)',
 1: 'Covasna (Kovászna)',
 2: 'Harghita (Hargita)',
 15: 'Hunedoara (Hunyad)',
 5: 'Maramureş (Máramaros)',
 3: 'Mureş (Maros)',
 7: 'Sălaj (Szilágy)',
 6: 'Satu Mare (Szatmár)',
 16: 'Sibiu (Szeben)',
 10: 'Timiş (Temes)'}

In [23]:
html = urlopen(url)
soup = BeautifulSoup(html)

county = []
for link in soup.findAll('a'):
    county.append(link.get('href'))

#kiszedjuk az egyes megyekhez tartozo linkeket
county_link = []
c = 0 
for l in county:
    if "/?pg=telepuleslista&megye=" in l:
        c = c + 1
        if (c % 3 == 0):
            county_link.append("http://nepszamlalas.adatbank.ro/"+ str(l))
print(county_link)

['http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=11', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=8', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=4', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=12', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=14', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=9', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=13', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=1', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=2', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=15', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=5', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=3', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=7', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=6', 'http://nepszamlalas.adatbank.ro//?pg=telepuleslista&megye=16', 'http://nepszamlalas.adatbank.ro//?pg=telepulesl

In [29]:
url = county_link[9]
html = urlopen(url)
soup = BeautifulSoup(html)
towns = []

for link in soup.findAll('a'):
    towns.append(link.get('href')) 
    
#kiszedjuk az egyes megyekhez tartozo telepulesek linkjeit
towns_links = []
for l in towns:
    if "/?pg=felekezeti&id=" in l:
        towns_links.append("http://nepszamlalas.adatbank.ro/"+ str(l))
        #print(l)
towns_links

['http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6113',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6117',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6114',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6115',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6116',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6228',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6229',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6230',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6232',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6145',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6146',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6164',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6165',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6166',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6167',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6168',
 'http://nepszamlalas.adatbank.ro//?pg=f

In [30]:
k = []
kk = []
for link in soup.findAll('span'):
    #k.append(link.get('style'))
    k.append(link.text)

for i in range(len(k)):
    if ("municípium" in str(k[i]) or "község" in str(k[i]) or "város" in str(k[i])):
          print(i-1, str(k[i]))
            
    if (str(k[i]) == 'None'):
        kk.append(i-1)
        print(i, str(k[i]))
main_towns = []
#kk = kk[1:]
#for i in range(len(kk)):
#    if (i %2 == 0):
#          main_towns.append(kk[i])

0 Déva municípium
10 Zsilyvajdejvulkán municípium
18 Brád municípium
32 Vajdahunyad municípium
46 Lupény municípium
50 Szászváros municípium
52 Szászváros
54 Petrozsény municípium
64 Szlatinova községrész
66 Aninoszabányatelep város
72 Pusztakalán város
100 Algyógy város
102 Algyógy község
124 Hátszeg város
134 Petrilla város
145 Piskitelep város
161 Hobicaurikány város
168 Bácsi község
178 Körösbánya község
198 Boica község
222 Balsa község
230 Bunesd községrész
232 Galbina községrész
249 Banica község
257 Nagybár község
267 Batrina község
274 Berény község
284 Alsóvárosvíz
291 Blezseny község
302 Bosoród község
317 Branyicska község
337 Oláhbrettye község
364 Bucsesd község
377 Bukuresd község
388 Bulzesd község
403 Bunyila község
415 Burzsuk község
429 Kersec község
441 Cserbel község
459 Felsőcsertés község
478 Kristyor község
487 Demsus község
503 Dobra község
531 Alsófarkadin község
543 Gyalár község
553 Guraszáda község
577 Haró község
587 Marosillye község
607 Alsólapugy község

In [31]:
#for i in range(len(main_towns)):
#    if (main_towns[i] %2 == 1):
#        for j in range(i, len(main_towns)):
#            main_towns[j] = main_towns[j] + 1
#print(main_towns)
#main_towns = [int(i/2) for i in main_towns]
#print("\n")
#print(main_towns)

In [32]:
#main_towns_link = []
#for i in main_towns:
#    main_towns_link.append(towns_links[i])
#    print(towns_links[i])
#print(len(main_towns_link))

In [33]:
htm_1850_1992 = []
for i in towns_links:
    html = urlopen(i)
    soup = BeautifulSoup(html)
    
    for link in soup.findAll('iframe'):
        htm_1850_1992.append(link.get('src'))
        
htm_1850_1992 = list(filter(None, htm_1850_1992))
#for i in range(len(htm_1850_1992)):
#    htm_1850_1992[i] = re.sub(r'\.\/','/',htm_1850_1992[i])
    
htm_1850_1992

['./megyek/15/felek/1.htm',
 './megyek/15/felek/1.htm',
 './megyek/15/felek/1.htm',
 './megyek/15/felek/1.htm',
 './megyek/15/felek/1.htm',
 './megyek/15/felek/13.htm',
 './megyek/15/felek/13.htm',
 './megyek/15/felek/13.htm',
 './megyek/15/felek/13.htm',
 './megyek/15/felek/5.htm',
 './megyek/15/felek/5.htm',
 './megyek/15/felek/5.htm',
 './megyek/15/felek/5.htm',
 './megyek/15/felek/5.htm',
 './megyek/15/felek/5.htm',
 './megyek/15/felek/5.htm',
 './megyek/15/felek/2.htm',
 './megyek/15/felek/2.htm',
 './megyek/15/felek/2.htm',
 './megyek/15/felek/2.htm',
 './megyek/15/felek/2.htm',
 './megyek/15/felek/2.htm',
 './megyek/15/felek/2.htm',
 './megyek/15/felek/8.htm',
 './megyek/15/felek/8.htm',
 './megyek/15/felek/9.htm',
 './megyek/15/felek/9.htm',
 './megyek/15/felek/3.htm',
 './megyek/15/felek/3.htm',
 './megyek/15/felek/3.htm',
 './megyek/15/felek/3.htm',
 './megyek/15/felek/3.htm',
 './megyek/15/felek/3.htm',
 './megyek/15/felek/4.htm',
 './megyek/15/felek/4.htm',
 './megyek/15/fe

In [68]:
link_dict = {}
for i in towns_links:
    html = urlopen(i)
    soup = BeautifulSoup(html)
    
    for link in soup.findAll('iframe'):
        link_dict[i] = re.sub(r'\.\/','/',link.get('src'))
        
link_dict

{'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6113': '/megyek/15/felek/1.htm',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6117': '/megyek/15/felek/1.htm',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6114': '/megyek/15/felek/1.htm',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6115': '/megyek/15/felek/1.htm',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6116': '/megyek/15/felek/1.htm',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6228': '/megyek/15/felek/13.htm',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6229': '/megyek/15/felek/13.htm',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6230': '/megyek/15/felek/13.htm',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6232': '/megyek/15/felek/13.htm',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6145': '/megyek/15/felek/5.htm',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&id=6146': '/megyek/15/felek/5.htm',
 'http://nepszamlalas.adatbank.ro//?pg=felekezeti&

In [69]:
htm_1850_1992 = set(htm_1850_1992)
htm_1850_1992= list(htm_1850_1992)
#htm_1850_1992

In [71]:
for j in htm_1850_1992:  
    
    url  = "http://nepszamlalas.adatbank.ro" + j
    resp = requests.get(url)
    if resp.status_code != 200:        
        print(url + ' Web site not exists')
    else:
        html = resp.content
        soup = BeautifulSoup(html)

        #az 1850 - 1992es nepszamlalalsi adatok span tagek koze vannak irva    
        span_tags = soup.find_all("span")
        #print(span_tags)

        #a linkbol kiszedni, hogy melyik megye
        county = url.split("/")[4]
        index = j.split("/")[4]
        index = str(re.sub(".htm","",index))
        county = county_index[int(county)]
        county_RO = county.split(" (")[0]
       # print(index)

        data = []
        for span in span_tags:
            data.append(span.text)

        for i in range(len(data)):
            data[i] = re.sub(r'\xa0', '', data[i])  
            data[i] = re.sub(r'\r\n', ' ', data[i]) 
            data[i] = re.sub(r'\([a-z]\)', ' ', data[i]) 
            data[i] = re.sub(r'[\(\)*]', ' ', data[i]) 
            data[i] = re.sub(r'  ',' ',data[i]) 
            data[i] = re.sub(r'  ',' ',data[i])

        data = list(filter(lambda x : x != ' ', data))
        data = list(filter(lambda x : x != '', data))
        for i in range(2,len(data)):
            if ("-" not in data[i] and (data[i][0].isalpha() or data[i][1].isalpha()) and data[i] != "Összesen" and data[i][-1] == "."):
                data[i] = ""
            if(data[i].startswith(' Az ')):
                data[i] = ""
            if(data[i].startswith(' A ')):
                data[i] = ""
            if("kimutatva" in data[i]) or ("Ebből" in data[i]) or ("visszakerült" in data[i]) or ("különvált" in data[i]) or ("beolvadt" in data[i]) or ("része" in data[i]) or ("alkotott" in data[i]) or ("település" in data[i]):
                data[i] = ""
        data = list(filter(lambda x : x != '', data))

        for i in range(len(data)):
            data[i] = re.sub(r'[0-9]+\. ','',data[i])
        for i in range(2,len(data)):
            kk = re.findall(r'[0-9]*\+[0-9]*',data[i])
            if kk != []:
                s = kk[0].split("+")
                s = int(s[0]) + int(s[1])
                k = np.str(s)
                data[i] = data[i].replace(kk[0], k)

        for i in range(len(data)):
            if (data[i] == "Összesen") or (data[i] == "összesen") :
                data[i] =  data[0].split(" - ")[0]

        filename = cleanAccent(county_RO) + index + ".csv"
        with open(filename, 'a') as f:

            f.write('Megye,Telepules,Ev,Osszes,Ortodox,GorogKatolikus,RomaiKatolikus,Reformatus,Evangelikus,Unitarius,Izraelita,Mas,Baptista,Punkosdista,Adventisa,Muzulman,Evangeliumi,OhituOrtodox,ZsinatPresb,AgostaiEvang,FelekezetenKivuli,Ateista,NemValaszolt,Jehova')
            f.write("\n")
            town_name = data[1]

            for line in data:
                if line and line[0].isalpha():
                    if town_name != line:
                        town_name = line
                        town_RO = cleanAccent(town_name.split(" - ")[0])
                        town_RO = re.sub(r' $','',town_RO)

                else:
                    x = line.split(" ")
                    data_csv = cleanAccent(county_RO) + "," + cleanAccent(town_RO)


                    for i in range(len(x)):
                        if(x[i] == "." or x[i] == "-"):
                             x[i] = "0"
                        data_csv = data_csv + "," + x[i]

                    for i in range(9):
                        data_csv = data_csv + "," + "0"
                    f.write("%s\n" % cleanAccent(data_csv))

In [72]:
rel_sequence_2002 = [0,1,5,2,3,13,9,15,16,6,4,7,8,10,11,12,14,17,18,19]
rel_sequence_2011 = [0,1,5,2,3,13,9,15,17,6,4,7,8,10,11,12,14,18,19,20,16]

for i in towns_links:
    if i in link_dict.keys():
        index = link_dict[i].split("/")[4]
        index = str(re.sub(".htm","",index))
        filename = cleanAccent(county_RO) + index + ".csv"

    url  = i
    table = pd.read_html(url)
    
    clean1 = np.delete(table[6].values,-1,0)
    religion_2011_2002 = np.delete(clean1,-1,0)
    
    religion_2011 = []
    religion_2002 = []
    if(religion_2011_2002.size == 90):
        religion_2011 = religion_2011_2002[:10]
        religion_2002 = religion_2011_2002[10:]
    else:
        if(religion_2011_2002.size == 40):
            if(str(table[6].axes[1][0][-4:]) == '2002'):
                religion_2002 = religion_2011_2002[:10]
            else:
                if(str(table[6].axes[1][0][-4:]) == '2011'):
                    religion_2011 = religion_2011_2002[:10]
                    
    html = urlopen(url)
    soup = BeautifulSoup(html)

    for link in soup.findAll('h4'):
        town_RO = link.text
    town_RO = cleanAccent(town_RO).split("/")[0]
    town_RO = town_RO[:-1]
    town_RO = cleanAccent(town_RO)
    if ("municipiul" in town_RO):
        town_RO = re.sub(r' municipiul','',town_RO)
        town_RO = "Municipiul " + town_RO
    if("o." in town_RO):
        town_RO = re.sub(r' o.','',town_RO)
        town_RO = "Orasul " + town_RO 
    if("oras" in town_RO):
        town_RO = re.sub(r' orasul',"",town_RO)
        town_RO = "Orasul " + town_RO  
    if("c." in town_RO):
        town_RO = re.sub(r' c.','',town_RO)
        town_RO = "Comuna " + town_RO  
    if("comuna" in town_RO):
        town_RO = re.sub(r' comuna','',town_RO)
        town_RO = "Comuna " + town_RO
        
    town_RO = town_RO.split("(")[0] 
    religion_data_list_2002 = []
    if len(religion_2002) != 0:
        for i in range(len(religion_2002)):
            if(i % 2 == 1): 
                religion_data_list_2002 = list(itertools.chain(religion_data_list_2002,religion_2002[i]))

        data_csv = cleanAccent(county_RO) + "," + cleanAccent(town_RO) + "," + "2002"

        for i in rel_sequence_2002:
            #print(i)
            item = religion_data_list_2002[i]
            if(item == "*" or item == "-"):
                item = "0"
            data_csv = data_csv + "," + str(item)
        #print(data_csv)
        data_csv = data_csv + ",0"
        with open(filename, 'a') as f:
            f.write("%s\n" % data_csv)
     
    religion_data_list_2011 = []
    if len(religion_2011) != 0:
        for i in range(len(religion_2011)):
            if(i % 2 == 1): 
                religion_data_list_2011 = list(itertools.chain(religion_data_list_2011,religion_2011[i]))
         
        data_csv = cleanAccent(county_RO) + "," + cleanAccent(town_RO) + "," + "2011"
        
        for i in rel_sequence_2011:
            item = religion_data_list_2011[i]
            if(item == "*" or item == "-"):
                item = "0"
            data_csv = data_csv + "," + str(item)
        with open(filename, 'a') as f:
            f.write("%s\n" % data_csv)